# Baseline Convolutional NN
This notebook will use a CNN model to classify images. This model will be a baseline, since I will also be bringing in a pretrained model (VGG-16) in a later notebook to create more accurate predictions.   

### Importing modules

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.utils.vis_utils import plot_model
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import numpy as np

Using TensorFlow backend.


### Preparing the data

In [2]:
# dimensions of the images
img_width, img_height = 178, 218

train_data_dir = 'data/train'
test_data_dir = 'data/test'
epochs = 5
batch_size = 16

### Topology of basic model

In [3]:
# instantiating model
model = Sequential()

# Adding a convolutional 2D layer with a a pooling layer
model.add(Conv2D(32, (3, 3), input_shape=[178,218,1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Adding a convolutional 2D layer with a a pooling layer
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Adding a convolutional 2D layer with a a pooling layer
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Adding a dense layer to flatten convolutional layers
model.add(Flatten())  
model.add(Dense(64))
model.add(Activation('relu'))

# Final dense layer to produce output
model.add(Dropout(0.5))
model.add(Dense(7))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', ])

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 176, 216, 32)      320       
_________________________________________________________________
activation_1 (Activation)    (None, 176, 216, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 88, 108, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 86, 106, 32)       9248      
_________________________________________________________________
activation_2 (Activation)    (None, 86, 106, 32)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 43, 53, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 41, 51, 64)        18496     
__________

In [29]:
# currently unable to print out the CNN topology
# plot_model(model, to_file='baseline_model_topology.png')

### Reading in data

In [4]:
# this is the augmentation configuration we will use for training
datagen = ImageDataGenerator()

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = datagen.flow_from_directory(
        directory=train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle = True,
        color_mode = 'grayscale')

# this is a similar generator, for test data
test_generator = datagen.flow_from_directory(
        directory=test_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle = False,
        color_mode = 'grayscale')

Found 7456 images belonging to 7 classes.
Found 2284 images belonging to 7 classes.


#### Fitting and running the model

In [5]:
model.fit_generator(
        train_generator,
        steps_per_epoch= 7456//batch_size,
        epochs=epochs)

Epoch 1/5
466/466 [==============================] - 236s 507ms/step - loss: 13.1638 - acc: 0.1817
Epoch 2/5
466/466 [==============================] - 244s 524ms/step - loss: 12.8865 - acc: 0.1994
Epoch 3/5
466/466 [==============================] - 263s 565ms/step - loss: 12.8355 - acc: 0.2024
Epoch 4/5
466/466 [==============================] - 254s 545ms/step - loss: 13.0037 - acc: 0.1886
Epoch 5/5
466/466 [==============================] - 272s 583ms/step - loss: 12.8040 - acc: 0.1943


### Evaluating the model

In [6]:
# evaluate the model
loss, acc = model.evaluate_generator(test_generator, steps=2284 // batch_size)
print('Cross-entropy: ', loss)
print('Accuracy: ', acc)

Cross-entropy:  12.570979359284253
Accuracy:  0.22007042253521128


### Saving the model and the weights

In [7]:
# serialize model to JSON
base_model_json = model.to_json()
with open("base_model.json", "w") as json_file:
    json_file.write(base_model_json)
    
# serialize weights to HDF5
model.save_weights("base_model.h5")
print("Saved model to disk")

Saved model to disk


The model is achieving at about 22% accuracy in the test (validation) data, which is the baseline accuracy score of guessing the most frequently occuring class. In this instance, it seems that the model is not learning anything from the data.
Next steps:
- Using a pre-trained model
- Doing some image editing (reducing pixel size, using facial landmarks)
- Looking at the other metrics (precision, recall, specificity) to determine where the model is incorrectly classifying

In [33]:
%%time
test_generator.reset()
pred=model.predict_generator(test_generator, np.ceil(2284/batch_size))

CPU times: user 59.1 s, sys: 14.7 s, total: 1min 13s
Wall time: 27.3 s


In [34]:
labels = (test_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in np.argmax(pred,axis=1)]

In [35]:
filenames=test_generator.filenames

In [36]:
classes = [x.split('/')[0] for x in filenames]

In [39]:
results=pd.DataFrame({"Filename":classes, "Predictions":predictions})

In [50]:
results.head()

,Filename,Predictions
0,0-10,31-40
1,0-10,31-40
2,0-10,31-40
3,0-10,31-40
4,0-10,31-40
